In [ ]:
!pip install kaggle

In [ ]:
!pip install py7zr

In [ ]:
from google.colab import files

files.upload()

In [ ]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [ ]:
! kaggle competitions download -c malware-classification -f train.7z

In [ ]:
# !7z e train.7z -o/content *.bytes -r

In [ ]:
import py7zr
with py7zr.SevenZipFile('train.7z', 'r') as zf:
  ls = zf.getnames()
  byt_f = [x for x in ls if x.endswith('.bytes')]

In [ ]:
with py7zr.SevenZipFile('train.7z', 'r') as zf:
    zf.extract(path="bytefiles", targets=byt_f)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import scipy.sparse as sp

In [ ]:
vectorizer = TfidfVectorizer(ngram_range=(2,2))
byte_bigram_data = sp.csr_matrix((1, 66049))

In [ ]:
file = open('bytefiles/' + byt_f[0])
corpus = [file.read().replace('\n', ' ').lower()] 
bigrams=vectorizer.fit_transform(corpus)
byte_bigram_data = sp.csr_matrix(bigrams)

In [ ]:
vectorizer = TfidfVectorizer(tokenizer=lambda x: x.split(),lowercase=False,ngram_range=(2,2)) 
byte_bigram_data=sp.csr_matrix((1, 66049))

file = open('bytefiles/' + byt_f[0])
corpus = [file.read().replace('\n', ' ').lower()] 
bigrams=vectorizer.fit_transform(corpus)
byte_bigram_data = sp.csr_matrix(bigrams)

for i in range(1,10868): 
    file = open('bytefiles/' + byt_f[i])
    corpus = [file.read().replace('\n', ' ').lower()] 
    bigrams=vectorizer.fit_transform(corpus)
    byte_bigram_data = sp.vstack((byte_bigram_data,bigrams))
    file.close()

In [ ]:
# you can extract only one set of files
# first try changing the run time to GPU so that you can get more hard disk
# download only train.7z file, if you have dowloaded all the files from the competiion you can remove other files except train.7z
# then as shown in the above cell you can extract only the byte files
# after you extract byte files
# for every bytefile:
#   create bigrams from that files, it should be stright forward for you, 
#   you can reuse the code in your tfidf assignment or you can use the tfidf vectorizer with fixed vocabulary
#   so after the computing of the bigrams it will be a 1*vocab size sparse matrix
#   delete the file extracted byte file

# you need to concatinate all the sparse matrices, which makes the final matrix of size (number_of_bytefiles, len_vocab)

# you can do the same thing for the asm files also, after you the above for loop, all the byte files might have been removed
# extract only asm files 
# for every asm file:
#   create pixel features, check the reference we have given
#   so after the computing of the pixel features it will be a 1*len_pixel_values a dense matrix
#   delete the file extracting asm file

In [ ]:
# Store both the sparse matrix, which holds the bigrams and the the dense matrix which holds the pixel matrix into pickle files so that you can reuse them later

# if you can identify each row of the matrix by a unique file name, that might be help you in train test split separation 
